In [1]:
import oursin as urchin
import pandas as pd
import numpy as np
from one.api import ONE
one = ONE(base_url='https://openalyx.internationalbrainlab.org', password='international', silent=True)
# one = ONE(base_url='https://alyx.internationalbrainlab.org', silent=False)
from reproducible_ephys_functions import *

import ibllib.atlas as atlas
CCFAtlas = atlas.AllenAtlas(25)
NeedlesAtlas = atlas.NeedlesAtlas(25)

import pandas as pd

In [3]:
def rgb_to_hex(rgb):
    return '#%02x%02x%02x' % rgb

In [4]:
# insertions = get_histology_insertions(one=one, freeze='biorxiv_2022_05')

insertions_11 = get_insertions(level=0, one=one, freeze='release_2022_11', recompute=False)






# todo before you run this: run the actual download script reproducible_ephys_run.py

In [31]:
temp = insertions_11[0]
temp['depth'] = 6000

In [37]:
# create one example optimal insertion
planned_ins = atlas.Insertion.from_dict(temp)
planned_mlapdv = list(NeedlesAtlas.xyz2ccf(planned_ins.tip))
planned_angles = [planned_ins.phi, planned_ins.theta, 0]

In [38]:
repro = pd.DataFrame(columns=['ml','ap','dv','phi','theta','depth'])

for i, insertion in enumerate(insertions_11):

    real_insertion = one.alyx.rest('trajectories', 'list', provenance='Ephys aligned histology track',probe_insertion=insertion['probe_insertion'])
    if len(real_insertion) > 0:
        real_insertion = real_insertion[0]
        ins = atlas.Insertion.from_dict(real_insertion)
        mlapdv = NeedlesAtlas.xyz2ccf(ins.tip)

        repro.loc[i] = [mlapdv[0], mlapdv[1], mlapdv[2], ins.phi, ins.theta, ins.depth]

In [8]:
repro.head()

,ml,ap,dv,phi,theta,depth
0,4125.941471,7595.473550,4896.364506,156.630500,8.767287,0.004474
1,4164.419198,8637.391136,4669.309954,137.428414,11.561574,0.004332
2,4577.826682,7756.631760,4623.343641,152.702252,15.598882,0.004333
3,4624.268572,9080.011132,4035.252208,135.518175,22.498909,0.003935
4,4345.399131,7710.907315,4645.161821,144.936123,14.305560,0.004304


In [9]:
urchin.setup()

(URN) connected to server
Login sent with ID: root


In [39]:
areas = ['VISa','CA1','DG','LP','PO']

area_vis = {}
area_mat = {}
area_alpha = {}

for area in areas:
  area = str(area)
  area_vis[area] = True
  area_mat[area] = 'transparent-unlit'
  area_alpha[area] = 0.4

urchin.clear()

# root area
urchin.ccf.set_visibility({'grey':True})
urchin.ccf.set_material({'grey':'transparent-unlit'})
urchin.ccf.set_color({'grey':'#d3d3d3'})
urchin.ccf.set_alpha({'grey':0.2})

# individual areas
urchin.ccf.set_visibility(area_vis)
urchin.ccf.set_material(area_mat)
urchin.ccf.set_alpha(area_alpha)

urchin.camera.set_rotation(60,0,150)
# urchin.camera.set_mode('perspective')

In [40]:
probeNames = []
probePos = {}
probeAngle = {}
probeColor = {}
probeMaterial = {}
probe_sizes = {}

for i,row in repro.iterrows():
    name = 'p'+str(i)
    probeNames.append(name)
    
    probePos[name] = [row['ml'],row['ap'],row['dv']]
    probeAngle[name] = [row['phi'],row['theta'],0]
    probeColor[name] = rgb_to_hex((75,75,75))
    
    probe_sizes[name] = [0.070, np.round(row['depth']*1000000)/1000, 0.020]

# add also the planned insertion
pname = 'planned'
probeNames.append(pname)
probePos[pname] = planned_mlapdv
probeAngle[pname] = planned_angles
probeColor[pname] = rgb_to_hex((255,0,0))
probe_sizes[pname] = [0.01, 8, 0.01]

In [41]:
urchin.clear_probes()
urchin.probes.create(probeNames)
urchin.probes.set_position(probePos)
urchin.probes.set_angle(probeAngle)
urchin.probes.set_color(probeColor)
urchin.probes.set_size(probe_sizes)